# Jack's Car Rental - Markov Decision Process
Jack has to decide each day how to move cars around his two locations, observing some operational contraints, in order to maximize expected monetary gains in the (not so) long term.

In [1]:
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
%matplotlib inline

## Problem and model parameters
Apart from the problem formulation (see slides), we assume that the following holds:
- Requests and returns are Poisson random variables, with specific parameters.
- No more than 7 cars can be moved each time.
- The model has to be discounted, with a specific discount factor.
- Each location can hold up to 30 cars, which gives us 30+1 possible conditions for each location (including "no cars"), thus (30+1)^2 states if we consider as state the number of cars in each location.
- The action is the number of cars moved around: we can move up to 7 cars, so 2*7+1 (including "no cars moved").

We assume that a state update, i.e. transition, happens when Jack decides what to do. Thus:
- The state is updated taking into account requests, returns, and cars moved by Jack (the "action"). This is the environment state signal.
- The action is performed afterwards: moving cars between the two locations.
- The reward has to be a difference between the expected gains and the costs.

In [2]:
# Float data type chosen for numerical precision.
# Set this to np.float128 to make very precise and computationally expensive calculations.
#prec = None
prec = np.float128

# Poisson random variables parameters.
lambda_req_1 = 3
lambda_req_2 = 4
lambda_ret_1 = 3
lambda_ret_2 = 2
# Model discount factor.
gamma = 0.9
# Monetary gain for each car returned (dollars).
ret_gain = 10.0
# Monetary cost of moving a car (dollars).
move_cost = 2.0
# Max cars at each location.
max_cars_1 = 30
max_cars_2 = 30
# Max number of moveable cars.
max_mov = 7
# Number of states.
S = (max_cars_1 + 1) * (max_cars_2 + 1)
# Number of actions.
A = (2 * max_mov) + 1

AttributeError: module 'numpy' has no attribute 'float128'

Given that the rate at which requests and returns happen is random, Poisson-distributed, we must think in terms of probabilities, for both rewards and state transitions.
Thus, to consider each possible combination, we need to fill:
- An _S\*S\*A_ transition probability matrix, that holds in each entry the probability to get from a state to another taking some action.
- An _S*A_ rewards matrix, that holds in each entry the reward gained by being in some state and taking some action.

In [3]:
# Transition probability matrix initialization.
P_trans = np.zeros((S, S, A), dtype=prec)
# Rewards matrix initialization.
R = np.zeros((S, A), dtype=prec)

Here comes the fun part: **probabilities of rentals and of returns do NOT depend on actions**, they just follow their own Poisson distributions without giving a damn about what Jack does. The only thing we know for sure is that rentals always take place **after** returns.
This makes state transition probability computation (i.e. _P\_trans_ filling) a tad bit harder than expected: we need to use the _law of total probability_, i.e. we need to consider for each action we can take:
- First, the probability of actually getting to some state, multiplying respective probabilities of returns and rentals that get us there.
- Then, the probability of moving the amount of cars that the action specifies, and this is either 1 or 0; but why? Because Jack can't move more than 7 cars around each time, so not all actions are possible in every state.

This is tricky stuff but gets the job done.
We need to define and fill three more matrices, with probabilities of _returns_, _rentals_ and _movements_ for each possible state and state/action configuration, as stated above.
The first two matrices, however, can't be built immediately since they're the product of _returns_ and _rentals_ probabilities for each of the two locations respectively, so we need four more matrices.
For convenience, we start from probability vectors, and then build the matrices.

In [4]:
# These hold possible numbers of cars (will save us some code later).
cars_1 = np.arange(0, max_cars_1 + 1, 1)
cars_2 = np.arange(0, max_cars_2 + 1, 1)

# Initialize returns probability vector for location 1.
prob_return_1 = np.zeros(max_cars_1 + 1, dtype=prec)
for i in range(max_cars_1):
    prob_return_1[i] = poisson.pmf(i, lambda_ret_1)
prob_return_1[max_cars_1] = 1.0 - np.sum(prob_return_1)  # This works as is 'cause last entry is still 0.

# Initialize returns probability vector for location 2.
prob_return_2 = np.zeros(max_cars_2 + 1, dtype=prec)
for i in range(max_cars_2):
    prob_return_2[i] = poisson.pmf(i, lambda_ret_2)
prob_return_2[max_cars_2] = 1.0 - np.sum(prob_return_2)  # This works as is 'cause last entry is still 0.

# Initialize rentals probability vector for location 1.
prob_rentals_1 = np.zeros(max_cars_1 + 1, dtype=prec)
for i in range(max_cars_1):
    prob_rentals_1[i] = poisson.pmf(i, lambda_req_1)
prob_rentals_1[max_cars_1] = 1.0 - np.sum(prob_rentals_1)  # This works as is 'cause last entry is still 0.

# Initialize rentals probability vector for location 2.
prob_rentals_2 = np.zeros(max_cars_2 + 1, dtype=prec)
for i in range(max_cars_2):
    prob_rentals_2[i] = poisson.pmf(i, lambda_req_2)
prob_rentals_2[max_cars_2] = 1.0 - np.sum(prob_rentals_2)  # This works as is 'cause last entry is still 0.

In [5]:
# WARNING: From now on, carefully remember that Python indexes stuff starting from zero.
# We'll convert state numbers into (cars_1, cars_2) configurations acting like we're converting indexes and
# subscripts for a 31x31 matrix, with number of cars for location 1 on the rows and for location 2 on the columns.
# We'll increment location 1 number of cars first, thus use indexes in a column-major fashion.

# Initialize return probability matrix for location 1.
# This tells the probability to go from a state "row" to a state "col", computed using the probability that
# the necessary amount of cars got back to location 1 only.
P_ret_1 = np.zeros((S, S), dtype=prec)
for s in range(S):
    # We're iterating on the states.
    # First, determine how many cars each location has in the current state.
    curr_cars = np.unravel_index(s, (max_cars_1 + 1, max_cars_2 + 1), order='F')
    # Now determine how many cars can be returned from this configuration, to location 1.
    # You have to evaluate all possibilities, up to a maximum each time.
    new_cars_1 = np.minimum(curr_cars[0] + cars_1, max_cars_1)
    # Now update the probability for each new state "s_p".
    # Note that s_p, where we end up, depends on where we were before and how many cars we got back,
    # so to each s_p we sum the probability to get back as many cars as are necessary to get there
    # from the current configuration.
    for i in range(len(new_cars_1)):
        s_p = np.ravel_multi_index((new_cars_1[i], curr_cars[1]), (max_cars_1 + 1, max_cars_2 + 1), order='F')
        P_ret_1[s, s_p] += prob_return_1[i]

# Initialize return probability matrix for location 2.
# This tells the probability to go from a state "row" to a state "col", computed using the probability that
# the necessary amount of cars got back to location 2 only.
P_ret_2 = np.zeros((S, S), dtype=prec)
for s in range(S):
    # We're iterating on the states.
    # First, determine how many cars each location has in the current state.
    curr_cars = np.unravel_index(s, (max_cars_1 + 1, max_cars_2 + 1), order='F')
    # Now determine how many cars can be returned from this configuration, to location 2.
    # You have to evaluate all possibilities, up to a maximum each time.
    new_cars_2 = np.minimum(curr_cars[1] + cars_2, max_cars_2)
    # Now update the probability for each new state "s_p".
    # Note that s_p, where we end up, depends on where we were before and how many cars we got back,
    # so to each s_p we sum the probability to get back as many cars as are necessary to get there
    # from the current configuration.
    for i in range(len(new_cars_2)):
        s_p = np.ravel_multi_index((curr_cars[0], new_cars_2[i]), (max_cars_1 + 1, max_cars_2 + 1), order='F')
        P_ret_2[s, s_p] += prob_return_2[i]

# Total return probability matrix: holds the probability to get from a state "row" to a state "col", computed
# using the probability that the necessary amounts of cars got, in every possible way, to the two locations.
P_ret = np.matmul(P_ret_1, P_ret_2, dtype=prec)

# Initialize rental probability matrix for location 1.
# This tells the probability to go from a state "row" to a state "col", computed using the probability that
# the necessary amount of cars get rented from location 1 only.
P_rent_1 = np.zeros((S, S), dtype=prec)
for s in range(S):
    # We're iterating on the states.
    # First, determine how many cars each location has in the current state.
    curr_cars = np.unravel_index(s, (max_cars_1 + 1, max_cars_2 + 1), order='F')
    # Now determine how many cars can be rented from this configuration, from location 1.
    # You have to evaluate all possibilities, down to a minimum of no cars each time.
    new_cars_1 = np.maximum(curr_cars[0] - cars_1, 0)
    # Now update the probability for each new state "s_p".
    # Note that s_p, where we end up, depends on where we were before and how many cars got rented,
    # so to each s_p we sum the probability to get rented as many cars as are necessary to get there
    # from the current configuration.
    for i in range(len(new_cars_1)):
        s_p = np.ravel_multi_index((new_cars_1[i], curr_cars[1]), (max_cars_1 + 1, max_cars_2 + 1), order='F')
        P_rent_1[s, s_p] += prob_rentals_1[i]

# Initialize rental probability matrix for location 2.
# This tells the probability to go from a state "row" to a state "col", computed using the probability that
# the necessary amount of cars get rented from location 2 only.
P_rent_2 = np.zeros((S, S), dtype=prec)
for s in range(S):
    # We're iterating on the states.
    # First, determine how many cars each location has in the current state.
    curr_cars = np.unravel_index(s, (max_cars_1 + 1, max_cars_2 + 1), order='F')
    # Now determine how many cars can be rented from this configuration, from location 2.
    # You have to evaluate all possibilities, down to a minimum of no cars each time.
    new_cars_2 = np.maximum(curr_cars[1] - cars_2, 0)
    # Now update the probability for each new state "s_p".
    # Note that s_p, where we end up, depends on where we were before and how many cars got rented,
    # so to each s_p we sum the probability to get rented as many cars as are necessary to get there
    # from the current configuration.
    for i in range(len(new_cars_2)):
        s_p = np.ravel_multi_index((curr_cars[0], new_cars_2[i]), (max_cars_1 + 1, max_cars_2 + 1), order='F')
        P_rent_2[s, s_p] += prob_rentals_2[i]

# Total rental probability matrix: holds the probability to get from a state "row" to a state "col", computed
# using the probability that the necessary amounts of cars got rented, in every possible way, from the two locations.
P_rent = np.matmul(P_rent_1, P_rent_2, dtype=prec)

# To test that we are doing right, row-sums should all be one.
if np.sum(P_ret.sum(axis=1)) != S:
    print("Shit happened in P_ret.")
    raise
if np.sum(P_rent.sum(axis=1)) != S:
    print("Shit happened in P_rent.")
    raise

# Initialize movement probability matrix.
# This holds the probability to get from a state "ros" to a state "col" taking the action
# that specifies the "submatrix".
# WARNING: We'll now map the 15 possible actions in [-7, 7]: "positive" action imply a movement
# from location 1 to location 2, whilst "negative" ones imply a movement from location 2 to
# location 1, both of |action| cars.
P_move = np.zeros((S, S, A), dtype=prec)
for s in range(S):
    # We're iterating on the states.
    # First, determine how many cars each location has in the current state.
    curr_cars = np.unravel_index(s, (max_cars_1 + 1, max_cars_2 + 1), order='F')
    for a in range(A):
        # Now we're iterating on the actions.
        # First, remap the current action.
        moved = a - max_mov
        # Then, compute how many cars you can really move from this configuration,
        # knowing how many cars there are in each location.
        actually_moved = np.max([-curr_cars[1], np.min([moved, curr_cars[0]])])
        # Finally, compute how many cars you end up in each location and set the corresponding
        # probability to 1 since this is a feasible action.
        new_cars_1 = np.min([curr_cars[0] - actually_moved, max_cars_1])
        new_cars_2 = np.min([curr_cars[1] + actually_moved, max_cars_2])
        s_p = np.ravel_multi_index((new_cars_1, new_cars_2), (max_cars_1 + 1, max_cars_2 + 1), order='F')
        P_move[s, s_p, a] = 1.0

# Now we can compute the definitive transition probability matrix.
# NOTE: This takes quite some time 'cause NumPy's BLAS implementation is
# not quite good yet.
for a in range(A):
    P_trans[:, :, a] = np.matmul(np.matmul(P_ret, P_rent, dtype=prec), P_move[:, :, a], dtype=prec)

Finally, we can store rewards in a matrix, as stated above.
To get there, we need to go by a few steps:
- First, compute the expected monetary earning given that a specific amount of cars have been returned to each location.
- Then, compute the expected monetary gain considering also the probability that, from any possible state, any possible amount of cars is returned to each location (i.e. the sum gets another dimension).
- At last, for each (_state_, _action_) pair, the reward will be the difference between the earning with the probability that makes it possible, given by _P\_ret_, and the cost of taking that action, i.e. moving that amount of cars.

In [6]:
# Now for the rewards, we need to compute what would be the earnings first,
# for each state.
earnings = np.zeros(S, dtype=prec)
for s in range(S):
    # We're iterating on the states.
    # First, determine how many cars each location has in the current state.
    curr_cars = np.unravel_index(s, (max_cars_1 + 1, max_cars_2 + 1), order='F')
    # How many cars can be rented from each location in this configuration?
    avail_1 = np.arange(curr_cars[0] + 1)
    avail_2 = np.arange(curr_cars[1] + 1)
    # Then, compute the probability that each possible rental configuration takes place.
    prob_rent_1 = np.copy(prob_rentals_1[0:curr_cars[0] + 1])
    prob_rent_2 = np.copy(prob_rentals_2[0:curr_cars[1] + 1])
    prob_rent_1[-1] = 1.0 - np.sum(prob_rent_1[0:-1])
    prob_rent_2[-1] = 1.0 - np.sum(prob_rent_2[0:-1])
    # The earning is an expected value.
    earnings[s] = ret_gain * (np.sum(np.multiply(avail_1, prob_rent_1, dtype=prec)) + np.sum(np.multiply(avail_2, prob_rent_2, dtype=prec)))

# Finally, we can fill the rewards matrix.
for a in range(A):
    moved = a - max_mov
    R[:, a] = np.matmul(P_ret, earnings, dtype=prec) - move_cost * np.abs(moved)

In [7]:
# Dump data on files.
P_trans.dump("jacks_P.dat")
R.dump("jacks_R.dat")